# Discussion 5

### Due Saturday November 7, 11:59:59PM


---

## Lecture Review

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import disc05 as disc

# Lecture Review: Missing Data

### Understanding Why Our Data are Missing

It is important to assess why our data are missing, as the observed data may not be representative of the complete dataset, nor the process that generated it. This limits the ability to draw conclusions about the process from the dataset.

#### Conditionally Ignorable / Missing at Random (MAR)

* Missing at Random means that the tendency for a data point to be missing *explainable* by some of the observed data.
    * Whether or not someone answered #1351 on your survey has nothing to do with the missing value (what the answer would've been), but it does have to do with the values of some other variable (e.g. whether it came near the beginning or end of a very long survey).
    * Think of this as (Conditionally) Missing at Random because the missingness is conditional on another observed variable.  
    * The idea is, if we can control for this conditional variable, we can get an unconditional random subset.
    
$$Pr({\rm data\ is\ present\ } | Y_{obs}, Y_{mis}, \psi) = Pr({\rm data\ is\ present\ } |\ Y_{obs}, \psi)$$

#### Unconditionally Ignorable / Missing Completely at Random (MCAR)

* The fact that a certain value is missing has nothing to do with its hypothetical value and with the values of other variables.
    * There is no relationship between whether a data point is missing and any values in the data set, missing or observed.

    * The missing data are just a random subset of the data.
    
$$Pr({\rm data\ is\ present\ } | Y_{obs}, Y_{mis}, \psi) = Pr({\rm data\ is\ present\ } |\ \psi)$$

#### Non-Ignorable / Missing Not at Random (NMAR)

* The missing value depends on the hypothetical (unobserved) value itself and isn't the tendency to be missing is'not explainable by any other data. 
    * e.g. People with high salaries generally do not want to reveal their incomes in surveys.
* Because non-ignorable missing data depends on the values of the unobserved data, you cannot determine it from the data alone! It is an assumption on the data generating process.
* If the you can add more attributes that 'explain' the missingness, non-ignorable data can become ignorable.
    * e.g. Adding Education level and zip code may explain the non-response of people with high-salaries.
    
$$Pr({\rm data\ is\ present\ }| Y_{obs}, Y_{mis}, \psi)$$

### Determining Missingness Types

Determining the mechanisms of missingness involves *searching for patterns to explain how data are missing*. When assessing a dataset:
1. In what ways might the missing values in a dataset come to be missing? Try to come up with explanations involving the data generating process. If you arrive at reasons for missingness that are not explainable by the dataset, then you have identified *non-ignorable* missing data. If this is the case, can you think attributes to collect that might explain the missing data?
1. Once you believe that patterns in the missingness of an attribute are explainable by the observed data, you can try to determine the nature of these patterns. In this case, look at the data when the attribute in question is missing, versus when it's not missing: do the data look the the same in both cases? To do this:
    - Consider the `missing` and `not_missing` data as two groups, and compare the distributions of the other attributes to determine if they look similar.
    - To determine the similarity of the two distributions, use a permutation test.
1. If the two distributions are different, then you've identified a pattern in the missingness: the likelihood the attribute is missing depends on the values of another attribute in the dataset (it's conditional on the other attribute).
1. If there are no patterns in the missingness of the data, then the dataset is *unconditionally ignorable*.

### Examples: Salaries

Below are the salaries of San Diego city employees, with missing data. Assuming that this data was collected via a survey given at work. (In reality, this dataset is a *census*, including every employees salaries).

**Question:** What are some potential explanations for why salary data might be missing from this dataset? What are potential fields that could be collected to explain the missingness?

In [ ]:
salaries1 = pd.read_csv('data/salaries_miss1.csv')
salaries2 = pd.read_csv('data/salaries_miss2.csv')
salaries1[['Total Pay']].head(7)

Next, we'll assess whether the 'Total Pay' is missing conditional on job 'Status'.

In [ ]:
df = salaries2
# df = salaries1

In [ ]:
df.head()

In [ ]:
df.isnull().mean().rename('proportion null').to_frame()

Compute the empirical distribution of 'Status' conditional on missingness of 'Total Pay'. Do these distributions look similar?

In [ ]:
distr = (
    df
    .assign(is_null=df['Total Pay'].isnull())
    .pivot_table(index='Status', columns='is_null', aggfunc='size')
    .apply(lambda x:x/x.sum())
)

distr.plot(kind='bar', title='Job Status conditional on missingness of Pay');

In [ ]:
distr

Is this difference between the distributions due to chance? Or are the distributions significantly different?

1. Measure similarity between the *observed* distributions with the TVD.
1. Run a permutation test to assess the magnitude of the observed statistic. Repeatedly:
    - The two groups are *missing pay* and *not missing pay*.
    - Shuffle the labels, separate the shuffled groups into 'random groups'.
    - Calculate the tvd of the random groups.
1. The distribution of the 'shuffled TVDs' is the null distribution. How likely was the observed TVD generated from the null distribution? Calculate a p-value!


In [ ]:
# observed TVD
obs = distr.diff(axis=1).abs().iloc[:,-1].sum() / 2
obs

In [ ]:
# Shuffle once
(
    df
    .assign(is_null=df['Total Pay'].isnull())  # create label column
    .assign(Status=df['Total Pay'].sample(frac=1, replace=False).reset_index(drop=True)) # shuffle 'total pay' -- same effect as shuffling label
    .pivot_table(index='Status', columns='is_null', aggfunc='size')  # compute counts for the conditional distribution
    .apply(lambda x:x/x.sum())  # normalize counts to a distribution
    .diff(axis=1).abs().iloc[:,-1].sum() / 2    # calculate the TVD of the shuffled distributions
)

In [ ]:
N = 1000

tvds = []
for _ in range(N):
    shuffed = (
        df
        .assign(is_null=df['Total Pay'].isnull())
        .assign(Status=df['Total Pay'].sample(frac=1, replace=False).reset_index(drop=True))
        .pivot_table(index='Status', columns='is_null', aggfunc='size')
        .apply(lambda x:x/x.sum())
        .diff(axis=1).abs().iloc[:,-1].sum() / 2
    )
    
    tvds.append(shuffed)

In [ ]:
pd.Series(tvds).plot(kind='hist', title='permutation test for missingness of Pay dependent on Status')
plt.scatter(obs, 0, c='r');

**Question:** Does the observed dataset well-represent, over-represent, or under-represent the mean salary of city employees?

In [ ]:
distr.plot(kind='bar', title='Distribution of Status conditional on missingness of Pay');

In [ ]:
df['Total Pay'].mean()

**Question:** How could you estimate the mean of the population with this understanding of missingness?

Below is the population mean from the complete dataset:

In [ ]:
salaries = pd.read_csv('data/salaries.csv')
salaries['Total Pay'].mean()

**Question:** What does this have to do with imputation? What if we want an imputed dataset with approximately the same mean as the population mean?

# Working with Missing Data in Pandas

* The way in which `Pandas` handles missing values is constrained by its reliance on the `NumPy` package, which does not have a built-in notion of `NaN` values for non-floating-point data types.
    * `Pandas` chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point `NaN` value, and the Python `None` object. 
    * This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

#### `None`: Pythonic Missing Data

* The first sentinel value used by Pandas is `None`, a Python singleton object that is often used for missing data in Python code. 
    * Because it is a Python object, `None` cannot be used in any arbitrary `NumPy`/`Pandas` array, but only in arrays with data type 'object' (i.e., arrays of Python objects).

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

* The use of Python objects in an array also means that if you perform aggregations like sum() or min() across an array with a None value, you will generally get an error:

In [ ]:
# this reflects the fact that addition between an integer and None is undefined
vals1.sum()

#### `NaN`: Missing Numerical Data

* The other missing data representation, `NaN` (acronym for Not a Number), is different; it is a special floating-point value.

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

* Notice that `NumPy` chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. 
* You should be aware that `NaN` is a bit like a data virus in that it infects any other object it touches. 
    * Regardless of the operation, the result of arithmetic with `NaN` will be another `NaN`.
    * Comparisons with `NaN` always return false.
    * Use the `pd.isnull` function to check if a single value is `NaN`

In [ ]:
1 + np.nan

In [ ]:
0 *  np.nan

In [ ]:
np.nan == np.nan

In [ ]:
np.nan > 0

In [ ]:
np.nan <= 0

In [ ]:
pd.isnull(np.nan)

* This means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful.

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

#### `NaN` and `None` in Pandas

* `NaN` and `None` both have their place, and `Pandas` is built to handle the two of them nearly interchangeably, converting between them where appropriate.

In [ ]:
pd.Series([1, np.nan, 2, None])

* For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. 
    * For example, if we set a value in an integer array to `np.nan`, it will automatically be upcast to a floating-point type to accommodate the NA.

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

#### Applying Pandas Methods to Columns with Null Values
* Unlike `NumPy`, methods applied to Pandas Series/DataFrame usually silently apply `dropna` before applying the operation.
* Below, compare the equivalent `Pandas` and `Numpy` methods

In [ ]:
df = pd.read_csv('data.csv')
df

In [ ]:
# Apply Pandas method `mean` to column 'B' (a Series)
df.loc[:, 'B'].mean()

In [ ]:
# Apply Numpy method `mean` to the np.ndarray underlying column 'B'.
df.loc[:, 'B'].values.mean()

**Discussion Question**
* What is the output of `df.loc[:, 'C'].values.min()`?
* What is the output of  `df.loc[:, 'C'].min()`?

In [ ]:
df
# ans1 = df.loc[:, 'C'].values.min()
# ans2 = df.loc[:, 'C'].min()

#### Imputing (Filling) Null Values

* Sometimes rather than dropping NA values, we'd rather replace them with a valid value. 
* This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good, non-null values. 
    * You could do this in-place using the `isnull()` method as a mask, but because it is such a common operation `Pandas` provides the `fillna()` method, which returns a copy of the array with the null values replaced.

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

In [ ]:
help(data.fillna)

In [ ]:
data.fillna(0)

In [ ]:
# forward-fill
data.fillna(method='ffill')

In [ ]:
# back-fill
data.fillna(method='bfill')

In [ ]:
fill_dict = dict(zip(list('abcde'), [1, 1.5, 2, 2.5, 3]))
fill_dict

In [ ]:
data.fillna(fill_dict)

In [ ]:
fill_values =  pd.Series([1, 1.5, 2, 2.5, 3], index=list('abcde'))
fill_values

In [ ]:
data.fillna(fill_values)

* For `DataFrames`, the options are similar, but needs additional data specified:
    1. Either an axis along which the fills take place, (for `method` options).
    2. Keys for which column to which to apply each dictionary/series.

In [ ]:
df

In [ ]:
# if a previous value is not available during a forward fill, the NA value remains
salaries2.fillna(method='ffill', axis=1)

In [ ]:
# Fill columns 0 and 3 with column 2; Fill column 1 with [10,20,30]
fill_dict = {
    'B': df.iloc[:, 3],
    'C': df.iloc[:, 4],
'D': pd.Series({13: 538, 15: 270})
}
df.fillna(fill_dict)

---
# Practice Problems

In [ ]:
dg = pd.read_csv('data.csv', index_col=0)

In [ ]:
dg.head()

In [ ]:
# check the proportion of null values per column


**Question 1**

Given the dataframe `dg`, impute (i.e. fill-in) the missing values of column `B` with the value of the index for that row. To do this, write a function `impute_with_index` that takes in a dataframe `dg` and outputs the imputed column `B` (a Series).

*Remark*: can you fill in *all* the columns this way?

**Question 2**

Given the dataframe `dg`, impute (i.e. fill-in) the missing values of each column using the last digit of the value of column `A`. For example, if a missing value for a given row has value 78 in column `A`, then fill in the missing value with the number 8.

Write your answer as a function `impute_with_digit` that takes in a dataframe `dg` and outputs the imputed dataframe.